<div align="right" style="text-align:right"><i>Peter Norvig<br>May 2015<br>Updated: 2020, 2025</i></div>

# When Cheryl Met Eve: A Birthday Story

The ***Cheryl's Birthday*** logic puzzle  [made the rounds](https://www.google.com/webhp?#q=cheryl%27s+birthday),
and  I wrote [code](Cheryl.ipynb) that solves it. In that notebook I said that one reason for solving the puzzle with code rather than pencil and paper is that you can do more with code.  

**[Gabe Gaster](http://www.gabegaster.com/)** proved me right when he [tweeted](https://twitter.com/gabegaster/status/593976413314777089/photo/1)  that he had used my code to generate a new list of dates that satisfies the constraints of the puzzle:

     January 15, January 4,
     July 13, July 24, July 30,
     March 13, March 24,
     May 11, May 17, May 30

In this notebook, I verify Gabe's result, and explore some other variations on the puzzle.

First, let's recap  [the original Cheryl's Birthday puzzle](https://en.wikipedia.org/wiki/Cheryl%27s_Birthday):

1. Albert and Bernard became friends with Cheryl, and want to know when her birthday is. Cheryl gives them a list of 10 possible dates:
   - May 15,     May 16,     May 19
   - June 17,    June 18
   - July 14,    July 16
   - August 14,  August 15,  August 17
3. **Cheryl** then privately tells Albert the month and Bernard the day of her birthday.
4. **Albert**: "I don't know when Cheryl's birthday is, and I know that Bernard does not know."
5. **Bernard**: "At first I didn't know when Cheryl's birthday is, but I know now."
6. **Albert**: "Then I also know when Cheryl's birthday is."
7. So when is Cheryl's birthday?

# Code for Original Cheryl's Birthday Puzzle

This is a slight modification of my [previous code](Cheryl.ipynb). The puzzle concerns these key concepts:

- **Possible dates** that might be Cheryl's birthday.
- **Knowing** which dates are still possible; knowing for sure when only one is possible.
- **Telling** Albert and Bernard specific facts about the birthday.
- **Statements** made by Albert or Bernard about their knowledge of the birthday.

I implement them as follows:
- The global variable `DATES` is a set of all possible dates (each date is a string).
- `know(possible_dates)` is a function that returns `True` when there is only one possible date.
- `told(part)` is a function that returns the set of possible dates that remain after Cheryl tells a part (month or day).
- A statement is a function; *statement*`(date)`  that returns true if the statement is true given that `date` is Cheryl's birthday.
- `satisfy(possible_dates, statement,...)` returns a subset of possible_dates for which all the statements are true.

In the [previous code](Cheryl.ipynb) I treated `DATES` as a constant, but in this version the whole point is exploring different sets of possible dates. The easiest way to refactor the code was to make `DATES` a global variable, and provide the function `set_dates` to set the value of the global variable. (It would be cleaner to package the dates into a non-global object, but it would be a big change to the  code to inject this all the way down to the function `told`, where it is needed.)

In [1]:
BeliefState = set # A set of possible values

DATES = BeliefState({'May 15', 'May 16', 'May 19', 'June 17', 'June 18', 
                     'July 14', 'July 16', 'August 14', 'August 15', 'August 17'})

def know(beliefs: BeliefState) -> bool:
    """A person `knows` the correct value if their belief state has only one possibility."""
    return len(beliefs) == 1

def month(date) -> str: return date.split()[0]
def day(date)   -> str: return date.split()[1]

def told(part: str) -> BeliefState:
    """Cheryl told a part of her birthdate to someone; return a belief state of possible dates."""
    return {date for date in DATES if part in date}

def cheryls_birthday(dates: BeliefState) -> BeliefState:
    """Return a subset of the dates for which all three statements are true."""
    return satisfy(set_dates(dates), albert1, bernard1, albert2)

def set_dates(dates: BeliefState) -> BeliefState: 
    """Set the global variable DATES to dates."""
    global DATES
    DATES = dates
    return dates

def satisfy(beliefs: BeliefState, *statements) -> BeliefState:
    """Return the subset of values in `beliefs` that satisfy all the statements."""
    return {value for value in beliefs if all(statement(value) for statement in statements)}

def albert1(date: str) -> bool:
    """Albert: I don't know when Cheryl's birthday is, and I know that Bernard does not know."""
    dates = told(month(date))
    return not know(dates) and not satisfy(dates, lambda date: know(told(day(date))))

def bernard1(date: str) -> bool:
    "Bernard: At first I don't know when Cheryl's birthday is, but I know now."
    at_first = told(day(date))
    now      = satisfy(at_first, albert1)
    return not know(at_first) and know(now)

def albert2(date: str) -> bool:
    "Albert: Then I also know when Cheryl's birthday is." 
    now = satisfy(told(month(date)), bernard1)
    return know(now)

In [2]:
# Some tests

assert month('May 19') == 'May'
assert day('May 19') == '19'

assert told('May') == {'May 15', 'May 16', 'May 19'} # Albert's belief state for May 15
assert told('15')  == {'August 15', 'May 15'}        # Bernard's belief state
assert not know(told('May'))                         # Albert does not know
assert not know(told('15'))                          # Bernard does not know

assert told('June') == {'June 17', 'June 18'}  # Albert's belief state for June 18
assert told('18') == {'June 18'}               # Bernard's belief state
assert not know(told('June'))                  # Albert does not know
assert know(told('18'))                        # Bernard DOES know

Below we trace through how this works.

First Albert says that he doesn't know, and that Bernard doesn't either. So the possible remaining dates are:

In [3]:
satisfy(DATES, albert1)

{'August 14', 'August 15', 'August 17', 'July 14', 'July 16'}

Bernard says he initially didn't know, but now he does. He knows, but we the puzzle-solvers don't. The remaining possible dates for us are:

In [4]:
satisfy(DATES, albert1, bernard1)

{'August 15', 'August 17', 'July 16'}

Now Albert knows, and so do we:

In [5]:
satisfy(DATES, albert1, bernard1, albert2)

{'July 16'}

In [6]:
assert cheryls_birthday(DATES) == {'July 16'}

# Verifying Gabe's Version

Gabe tweeted these ten dates:

In [7]:
gabe_dates = [
  'January 15', 'January 4',
  'July 13',    'July 24',   'July 30',
  'March 13',   'March 24',
  'May 11',     'May 17',    'May 30']

We can verify that they do indeed make the puzzle work:

In [8]:
assert cheryls_birthday(gabe_dates) == {'July 30'}
assert know(cheryls_birthday(gabe_dates))

# Creating Our Own Versions

If Gabe can do it, we can do it!  Our strategy will be to repeatedly pick a random sample of dates, and check if they solve the puzzle. We'll limit ourselves to a subset of dates (not all 366) to make it more likely that a random selection will have multiple dates with the same month and day (otherwise Albert and/or Bernard would know right away):

In [9]:
many_dates = {mo + ' ' + tens + ones
              for mo in {'April', 'August', 'July', 'June', 'March', 'May'}
              for tens in '12'
              for ones in '34567890'}

len(many_dates)

96

Now we need to cycle through random samples of these possible dates until we hit one that works.  I anticipate wanting to solve other puzzles besides the original `cheryls_birthday`, so I'll define  the function `pick_dates` to take a parameter, `puzzle`, and keep trying `puzzle(random_dates)` until the result is something we `know`:

In [10]:
import random

def pick_dates(puzzle, k=10) -> BeliefState:
    """Pick a set of `k` dates for which the `puzzle` has a unique solution."""
    while True:
        random_dates = set(random.sample(many_dates, k))
        solutions = puzzle(random_dates)
        if know(solutions):
            return random_dates

In [11]:
pick_dates(cheryls_birthday)

{'April 17',
 'April 18',
 'August 20',
 'June 13',
 'June 16',
 'June 26',
 'June 28',
 'March 17',
 'March 18',
 'March 20'}

In [12]:
pick_dates(cheryls_birthday, k=6)

{'July 13', 'March 13', 'March 16', 'March 19', 'May 16', 'May 19'}

Great! We can make a new puzzle, just like Gabe.  But how often do we get a unique solution to the puzzle (that is, the puzzle returns a set of size 1)?  How often do we get a solution where Albert and Bernard know, but we the puzzle solver don't know (that is, a belief set of size greater than 1)?  How often is there no solution (a set of size 0)? Let's make a Counter of the number of times each length-of-solution occurs:

In [13]:
from collections import Counter

def solution_lengths(puzzle, N=10000, k=10, many_dates=many_dates):
    """Try N random samples of k dates and count how often each possible 
    length-of-puzzle-solution appears."""
    return Counter(len(puzzle(random.sample(many_dates, k)))
                   for _ in range(N))

In [14]:
solution_lengths(cheryls_birthday)

Counter({0: 9572, 1: 153, 2: 275})

This says that a bit over 1% of the time we get a unique solution (a set of length 1). More often than that we get an ambiguous solution (with 2 or more possible birth dates), and about 95% of the time the sample of dates has no solution (a set of length 0).

What happens if Cheryl changes the number of possible dates?

In [15]:
solution_lengths(cheryls_birthday, k=6, N=100_000)

Counter({0: 99926, 2: 54, 1: 20})

In [16]:
solution_lengths(cheryls_birthday, k=12)

Counter({0: 9201, 1: 352, 2: 441, 3: 6})

With 6 dates, we get no solution 99.9% of the time, but with 12 dates, finding a set of dates with a unique solution is easy.

# A New Puzzle: All About Eve

Now let's see if we can create a more complicated puzzle. We'll introduce a new character, Eve, and keep the same puzzle as before, except that after Albert's second statement, Eve makes this statement:

- **Eve**: "Hi, Everybody. My name is Eve and I'm an evesdropper. It's what I do! I peeked and saw the first letter of the month and the first digit of the day. When I peeked, I didn't know Cheryl's birthday, but after listening to Albert and Bernard I do.  And it's a good thing I peeked, because otherwise I couldn't have
figured it out."

We can easily code this up:

In [17]:
def cheryls_birthday_with_eve(dates):
    "Return a set of the dates for which Albert, Bernard, and Eve's statements are true."
    return satisfy(set_dates(dates), albert1, bernard1, albert2, eve1)

def eve1(date):
    """Eve: I peeked and saw the first letter of the month and the first digit of the day. 
    When I peeked, I didn't know Cheryl's birthday, but after listening to Albert and Bernard 
    I do. And it's a good thing I peeked, because otherwise I couldn't have figured it out."""
    at_first = told(first_character(day(date))) & told(first_character(month(date)))
    return (not know(at_first) and
                know(satisfy(at_first,  albert1, bernard1, albert2)) and
            not know(satisfy(DATES, albert1, bernard1, albert2)))

def first_character(part: str) -> str: return part[0]

*Note*: I admit I "cheated" a bit here.  Remember that the function `told`  tests for `(part in date)`.  For that to work for Eve, we have to make sure that the first letter is distinct from any other character in the date (it is&mdash;because only the first letter is uppercase) and that the first digit is distinct from any other character (it is&mdash;because in `many_dates` I carefully made sure that the first digit is always 1 or 2, and the second digit is never 1 or 2). 

I have no idea if it is possible to find a set of dates that works for this puzzle. But I can try:

In [18]:
pick_dates(cheryls_birthday_with_eve)

{'April 16',
 'April 23',
 'August 24',
 'August 25',
 'June 18',
 'June 23',
 'June 28',
 'March 15',
 'May 16',
 'May 24'}

In [19]:
cheryls_birthday_with_eve(_)

{'April 23'}

That was easy.  How often is a random sample of dates a solution to this puzzle?

In [20]:
solution_lengths(cheryls_birthday_with_eve)

Counter({0: 9773, 2: 119, 1: 108})

A unique solution (a set of length 1) occurs about 1% of the time.

# An Even More Complex Puzzle

Let's make the puzzle even more complicated by making Albert wait one more time before he finally knows:

- Albert and Bernard just became friends with Cheryl, and they want to know when her birtxhday is. Cheryl wrote down a list of 10 possible dates for all to see.
- **Cheryl** then writes down the month and shows it just to Albert, and also writes down the day and shows it just to Bernard.
- **Albert**: I don't know when Cheryl's birthday is, but I know that Bernard does not know either. 
- **Bernard**: At first I didn't know when Cheryl's birthday is, but I know now.
- **Albert**: I still don't know.
- **Eve**: Hi, Everybody. My name is Eve and I'm an evesdropper. It's what I do! I peeked and saw the first letter of the month and the first digit of the day. When I peeked, I didn't know Cheryl's birthday, but after listening to Albert and Bernard I do.  And it's a good thing I peeked, because otherwise I couldn't have
figured it out.
- **Albert**: OK, now I know.
- So when is Cheryl's birthday?

Albert's second statement is different; he has a new third statement; and Eve's statement uses the same words, but it now implicitly refers to a different statement by Albert. We'll use the names `albert2c`,  `albert3c`, and `eve1c` (`c` for "complex") to represent the new statements:

In [21]:
def cheryls_birthday_complex(dates):
    "Return a set of the dates for which Albert, Bernard, and Eve's statements are true."
    return satisfy(set_dates(dates), albert1, bernard1, albert2c, eve1c, albert3c)

def albert2c(date):
    "Albert: I still don't know."
    return not know(satisfy(told(month(date)), bernard1))

def eve1c(date):
    """Eve: I peeked and saw the first letter of the month and the first digit of the day. 
    When I peeked, I didn't know Cheryl's birthday, but after listening to Albert and Bernard 
    I do. And it's a good thing I peeked, because otherwise I couldn't have figured it out."""
    at_first = told(day(date)[0]) & told(month(date)[0])
    return (not know(at_first)
            and know(satisfy(at_first, albert1, bernard1, albert2c)) and
            not know(satisfy(DATES, albert1, bernard1, albert2c)))

def albert3c(date):
    "Albert: OK, now I know."
    return know(satisfy(told(month(date)), bernard1, eve1c))

Again, I don't know if it is possible to find dates that works with this story, but I can try:

In [22]:
pick_dates(cheryls_birthday_complex)

{'April 28',
 'August 16',
 'August 19',
 'August 28',
 'July 10',
 'July 16',
 'July 20',
 'June 14',
 'June 19',
 'March 18'}

In [23]:
cheryls_birthday_complex(_)

{'August 28'}

It worked!  Were we just lucky, or are there many sets of dates that work?

In [24]:
solution_lengths(cheryls_birthday_complex)

Counter({0: 9386, 1: 613, 2: 1})

Interesting. It was actually easier to find dates that work for this story than for any of the other stories.

## Analyzing a Solution to the Complex Puzzle

Now we will go through a solution step-by-step.  We'll use these dates:

In [25]:
complex_dates = {
  'April 28',
  'July 27',
  'June 19',
  'June 16',
  'July 15',
  'April 15',
  'June 29',
  'July 16',
  'May 24',
  'May 27'}

Let's find the solution:

In [26]:
cheryls_birthday_complex(complex_dates)

{'July 27'}

Now the first step is that Albert was told "July":

In [27]:
told('July')

{'July 15', 'July 16', 'July 27'}

And no matter which of these three dates is the actual birthday, Albert knows that Bernard would not know the birthday, because each of the days (15, 16, 27) appears twice in the list of possible dates.

In [28]:
not know(told('15')) and not know(told('16')) and not know(told('27'))

True

Meanwhile, Bernard is told the day:

In [29]:
told('27')

{'July 27', 'May 27'}

There are two dates with a 27, so Bernard did not know then. But only one of these dates is still consistent after hearing Albert's statement:

In [30]:
satisfy(told('27'), albert1)

{'July 27'}

So after Albert's statement, Bernard knows. Poor Albert still doesn't know (after being told `'July'` and hearing Bernard's statement):

In [31]:
satisfy(told('July'), bernard1)

{'July 15', 'July 16', 'July 27'}

Then along comes Eve. She evesdrops the "J" and the "2":

In [32]:
told('J') & told('2')

{'July 27', 'June 29'}

Two dates, so Eve doesn't know after evesdropping. But only one of the dates works after hearing the three statements made by Albert and Bernard:

In [33]:
satisfy(told('J') & told('2'), albert1, bernard1, albert2c)

{'July 27'}

But Eve wouldn't have known if she hadn't evesdropped:

In [34]:
satisfy(DATES, albert1, bernard1, albert2c)

{'July 15', 'July 16', 'July 27'}

What about Albert?  After hearing Eve's statement he finally knows:

In [35]:
satisfy(told('July'), eve1c)

{'July 27'}

_______

# New Puzzle: Steve's Bus

Here's [another puzzle](https://www.reddit.com/r/riddles/comments/fw7h42/a_riddle_i_couldnt_solve/) that seems to have a very similar format:

1. Steve tells Alice the hour of his bus departure and he tells Annie at which minute it leaves. He also tells them both that the bus leaves between 06:00 and 10:00.
2. Alice and Annie consult the timetable and find the following services between those two time:
   - 06:32, 06:43, 06:50, 07:17, 07:46, 08:19, 08:32, 09:17, 09:19, 09:50.
4. Alice then says “I don’t know when Steve’s bus leaves but I am sure that neither does Annie”
5. Annie Replies “I didn’t know his bus, but now I do”
6. Alice responds “Now I do as well!”
7. When is Steve’s bus?

Upon closer inspection, not only is it a similar format, it is **exactly** the same puzzle, except that months are changed to hours and days to minutes.  If we change the colons in the times to spaces, we can solve the problem without changing the `cheryls_birthday` function:

In [36]:
times = '06:32, 06:43, 06:50, 07:17, 07:46, 08:19, 08:32, 09:17, 09:19, 09:50'.replace(':', ' ').split(', ')
cheryls_birthday(times)

{'08 32'}

Steve took the 8:32 bus.

_________

# Another New Puzzle: Evil Mad Scientist Cheryl

![](https://norvig.com/images/cheryl-trolley.png)

Again, we can solve this problem just by passing in a different set of "dates":

In [37]:
cheryls_birthday({'A 2', 'A 3', 'A 6', 'B 4', 'B 5', 'C 1', 'C 3', 'D 1', 'D 2', 'D 4'})

{'C 3'}

The correct pad is "C 3". 

(But may I point out that this Cheryl is not actually a mad scientist, just a [mad engineer](https://www.evilmadscientist.com/2015/evil-mad-engineers/). A true mad scientist would kill 25 people and use the other 25 as a control group.)

_______

# Yet Another Puzzle: Sum and Product

- A pair of numbers are chosen at random. They are both positive integers smaller than 100.
- Sam is told the sum of the numbers, while Pat is told the product of the numbers.
- Then, this dialog occurs:
- 1) Pat: I don't know the numbers.
- 1) Sam: I don't know the numbers.
- 2) Pat: I don't know the numbers.
- 2) Sam: I don't know the numbers.
- 3) Pat: I don't know the numbers.
- 3) Sam: I don't know the numbers.
- 4) Pat: I don't know the numbers.
- 4) Sam: I don't know the numbers.
- 5) Pat: I don't know the numbers.
- 5) Sam: I don't know the numbers.
- 6) Pat: I don't know the numbers.
- 6) Sam: I don't know the numbers.
- 7) Pat: I don't know the numbers.
- 7) Sam: I don't know the numbers.
- 8) Pat: Finally, I know the numbers.

What are the two numbers? In this version of the puzzle, each person has 7 iterations of "I don't know." What could the two numbers be if there are anywhere from 2 to 10 iterations? 

I could solve this using our existing `told` and `know` functions, but that would require encoding the pair of numbers, their sum, and their product into a string. So I'll introduce new functions instead:

In [38]:
from collections import Counter
from itertools   import combinations
from typing      import Dict
from math        import prod

def sum_and_product(numbers=range(1, 100), N=8) -> Dict[int, BeliefState]:
    """Solves the Sum and Product puzzle for various numbers of repetitions of Pat and Sam saying 
    "I don't know" (up to `N` times) and for a given range of integers.
    Returns a dictionary of {repetitions: {(x, y), ...}}."""
    pairs = set(combinations(numbers, 2))
    solutions = {}
    for i in range(2, N + 1):
        pairs = unknown_pairs(prod, pairs)      # Pat doesn't know
        pairs = unknown_pairs(sum, pairs)       # Sam doesn't know
        solutions[i] = known_pairs(prod, pairs) # Pat knows
    return solutions
        
def known_pairs(function, pairs: BeliefState) -> BeliefState:
    """Pairs for which function(pair) has a unique value."""
    counter = Counter(map(function, pairs))
    return {pair for pair in pairs if counter[function(pair)] == 1}

def unknown_pairs(function, pairs: BeliefState) -> BeliefState:
    """Pairs for which function(pair) does not have a unique value."""
    counter = Counter(map(function, pairs))
    return {pair for pair in pairs if counter[function(pair)] != 1}

sum_and_product()

{2: {(1, 6), (1, 8), (72, 92), (75, 96)},
 3: {(81, 88)},
 4: {(77, 90)},
 5: {(76, 90)},
 6: {(80, 84)},
 7: {(77, 84)},
 8: set()}

In [39]:
sum_and_product(range(2, 100))

{2: {(72, 92), (75, 96)},
 3: {(81, 88)},
 4: {(77, 90)},
 5: {(76, 90)},
 6: {(80, 84)},
 7: {(77, 84)},
 8: set()}

_______

# What Next?

There are many other directions you could take this:

- Could you create a Cheryl-puzzle that goes one or two rounds more before everyone knows?
- Could you add new characters: Faith, and then George, and maybe even a new Hope?
- Should we include the year or the day of the week, as well as the month and day?
- Perhaps a puzzle that starts with [Richard Smullyan](http://en.wikipedia.org/wiki/Raymond_Smullyan) announcing that one of the characters is a liar.
- Or you could make a puzzle harder than [the hardest logic puzzle ever](https://en.wikipedia.org/wiki/The_Hardest_Logic_Puzzle_Ever).
- Try the "black and white hats" [Riddler Express](https://fivethirtyeight.com/features/can-you-solve-these-colorful-puzzles/) stumper.
- It's up to you ...